## Statsmodel Forecast AB Testing

A/B  Testing is one method of models against each other.  This demonstration will show how to use the Wallaroo pipeline step `add_random_split` and `replace_with_random_split` to randomly submit inference input data into control and challenger models.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## Shadow Deploy

### Import Libraries

The first step is to import the libraries that we will need.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

from resources import simdb
from resources import util

pd.set_option('display.max_colwidth', None)

In [2]:
display(wallaroo.__version__)

'2023.2.1'

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, the `suffix` variable is generated from a random set of 4 ASCII characters.  To use the same workspace across the tutorial notebooks, hard code `suffix` and verify the workspace name created is is unique across the Wallaroo instance.

In [4]:
# used for unique connection names

suffix='-jch2'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload Model

The Python model created in "Forecast and Parallel Infer with Statsmodel: Model Creation" will now be uploaded.  Note that the Framework and runtime are set to `python`.

In [6]:
# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
control_model_file = './forecast_standard.py'

challenger01_model_name = 'forecast-challenger01-model'
challenger01_model_file = './forecast_alternate01.py'

challenger02_model_name = 'forecast-challenger02-model'
challenger02_model_file = './forecast_alternate02.py'

# upload the models

control_model = wl.upload_model(control_model_name, control_model_file, Framework.PYTHON).configure(runtime="python")

challenger_model_01 = wl.upload_model(challenger01_model_name, challenger01_model_file, Framework.PYTHON).configure(runtime="python")

challenger_model_02 = wl.upload_model(challenger02_model_name, challenger02_model_file, Framework.PYTHON).configure(runtime="python")


### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.  The pipeline configuration will allow for multiple replicas of the pipeline to be deployed and spooled up in the cluster.  Each pipeline replica will use 0.25 cpu and 512 Gi RAM.

In [7]:
# Set the deployment to allow for additional engines to run
pipeline.clear()
pipeline.add_model_step(control_model)
pipeline.deploy()

name,forecast-workshop-pipeline
created,2023-07-26 19:38:56.059951+00:00
last_updated,2023-07-26 20:59:02.808182+00:00
deployed,True
tags,
versions,"276d2c5e-a96f-48e6-8158-a18a68e3bd59, 5bf788b1-fa77-4fdb-a837-409af9c19e2d, 4b529632-8f30-4ae6-a588-8bca3971f20f, 659821ab-d0ad-40c2-984f-5052757c0782, 8dcf0175-07a3-4df3-8289-7b950a1de32b"
steps,forecast-challenger02-model


### Run Inference
For this example, we will forecast bike rentals by looking back one month from "today" which will be set as 2011-02-22.  The data from 2011-01-23 to 2011-01-27 (the 5 days starting from one month back) are used to generate a forecast for what bike sales will be over the next week from "today", which will be 2011-02-23 to 2011-03-01.

In [8]:
inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Replace Pipeline Step with Random Step

A 2:1:1 weighted random split - control will get 50% of the inference requests, the other two models 25% each.

In [9]:
pipeline.replace_with_shadow_deploy(0, control_model, [challenger_model_01, challenger_model_02])
pipeline.deploy()

name,forecast-workshop-pipeline
created,2023-07-26 19:38:56.059951+00:00
last_updated,2023-07-26 20:59:22.730271+00:00
deployed,True
tags,
versions,"b3579740-32b9-4b79-85ae-be71839c5d11, 276d2c5e-a96f-48e6-8158-a18a68e3bd59, 5bf788b1-fa77-4fdb-a837-409af9c19e2d, 4b529632-8f30-4ae6-a588-8bca3971f20f, 659821ab-d0ad-40c2-984f-5052757c0782, 8dcf0175-07a3-4df3-8289-7b950a1de32b"
steps,forecast-challenger02-model


In [10]:
inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

import time
import datetime

start_time = datetime.datetime.now()

time.sleep(5)

results = pipeline.infer(inferencedata)

end_time = datetime.datetime.now()

display(results)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

We can see this by looking at the pipeline logs and using the `metadata` filter to retrieve the logs.

The original output is listed by `out.{variable_name}`, with the shadow outputs as `out_{model.variable_name}`.  For this example, this is `out.json`, `out_forecast-challenger01-model.json`, etc.  The command below will filter for only the time and the output variables.

In [38]:
logs = pipeline.logs()
display(logs.filter(regex = 'time|out.*'))

Pipeline log schema has changed over the logs requested 2 newest records retrieved successfully, newest record seen was at <datetime>. Please request additional records separately


,time,out.json,out_forecast-challenger01-model.json,out_forecast-challenger02-model.json
0,2023-07-26 20:59:34.001,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}","{""forecast"":[1703,1757,1737,1744,1742,1743,1742]}","{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}"
1,2023-07-26 20:59:28.528,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}","{""forecast"":[1703,1757,1737,1744,1742,1743,1742]}","{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}"


### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [39]:
pipeline.undeploy()

name,forecast-workshop-pipeline
created,2023-07-26 19:38:56.059951+00:00
last_updated,2023-07-26 20:59:22.730271+00:00
deployed,False
tags,
versions,"b3579740-32b9-4b79-85ae-be71839c5d11, 276d2c5e-a96f-48e6-8158-a18a68e3bd59, 5bf788b1-fa77-4fdb-a837-409af9c19e2d, 4b529632-8f30-4ae6-a588-8bca3971f20f, 659821ab-d0ad-40c2-984f-5052757c0782, 8dcf0175-07a3-4df3-8289-7b950a1de32b"
steps,forecast-challenger02-model
